In [ ]:
'''
This code belongs to Miguel Pérez Cerdán. Should you have any comment or suggestion please feel free to contact
me through my email address: miguelpecerdan@gmail.com.

I will be updating the code periodically with new data and analysis. You can use it for your own studies as well.
'''

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import time
import csv
import pathlib
import os


# Constantes que vamos a usar durante todo el estudio.

In [2]:
# Estaciones según numeración del ayuntamiento
estaciones = [4, 8, 11, 16, 17, 18, 24, 27, 35, 36, 38, 39, 40, 47, 48,
              49, 50, 54, 55, 56, 57, 58, 59, 60]

# Nombres correspondientes a cada una de las estaciones
estaciones_nombres = ['Plaza España', 'Escuelas Aguirre',
                      'Av. Ramón y Cajal', 'Arturo Soria',
                      'Villaverde Alto', 'Calle Farolillo',
                      'Casa de Campo', 'Barajas', 'Pza. del Carmen',
                      'Moratalaz', 'Cuatro Caminos',
                      'Barrio del Pilar', 'Vallecas', 'Méndez Álvaro',
                      'Pº. Castellana', 'Retiro', 'Pza. Castilla',
                      'Ensanche Vallecas', 'Urb. Embajada (Barajas)',
                      'Plaza Elíptica', 'Sanchinarro', 'El Pardo',
                      'Parque Juan Carlos I', 'Tres Olivos']

# Diccionario de las estaciones con su código de distrito correspondiente
dict_estaciones = {'Plaza España': '09', 'Escuelas Aguirre': '04',
                   'Av. Ramón y Cajal': '05',
                   'Arturo Soria': '15', 'Villaverde Alto': '17',
                   'Calle Farolillo': '11', 'Casa de Campo': '09',
                   'Barajas': '21', 'Pza. del Carmen': '01',
                   'Moratalaz': '14', 'Cuatro Caminos': '07',
                   'Barrio del Pilar': '08', 'Vallecas': '13',
                   'Méndez Álvaro': '02', 'Pº. Castellana': '05',
                   'Retiro': '03', 'Pza. Castilla': '06',
                   'Ensanche Vallecas': '18',
                   'Urb. Embajada (Barajas)': '21',
                   'Plaza Elíptica': '12', 'Sanchinarro': '16',
                   'El Pardo': '08', 'Parque Juan Carlos I': '21',
                   'Tres Olivos': '08'}

# Coordenadas geográficas de las estaciones
coordenadas = {
    'Estacion': ['Plaza España', 'Escuelas Aguirre', 'Av. Ramón y Cajal',
                 'Arturo Soria', 'Villaverde Alto', 'Calle Farolillo',
                 'Casa de Campo', 'Barajas', 'Pza. del Carmen',
                 'Moratalaz',
                 'Cuatro Caminos', 'Barrio del Pilar', 'Vallecas',
                 'Méndez Álvaro', 'Pº. Castellana', 'Retiro',
                 'Pza. Castilla',
                 'Ensanche Vallecas', 'Urb. Embajada (Barajas)',
                 'Plaza Elíptica', 'Sanchinarro', 'El Pardo',
                 'Parque Juan Carlos I', 'Tres Olivos'],
    'Longitud': ['3°42\'43.91"O', '3°40\'56.22"O', '3°40\'38.5"O',
                 '3°38\'21.17"O', '3°42\'47.89"O', '3°43\'54.61"O',
                 '3°44\'50.44"O', '3°34\'48.10"O', '3°42\'11.4"O',
                 '3°38\'43.02"O', '3°42\'25.64"O', '3°41\'41.53"O',
                 '3°39\'5.5"O', '3°41\'12.57"O', '3°41\'25.34"O',
                 '3°40\'57.3"O', '3°41\'19.48"O', '3°36\'43.7"O',
                 '3°34\'50.03"O', '3°43\'7.54"O', '3°39\'37.86"O',
                 '3°46\'28.62"O', '3°36\'32.66"O', '3°41\'23.03"O'],
    'Latitud': ['40°25\'25.98"N', '40°25\'17.63"N', '40°27\'5.29"N',
                '40°26\'24.20"N', '40°20\'49.74"N', '40°23\'41.22"N',
                '40°25\'9.69"N', '40°28\'36.93"N', '40°25\'9.15"N',
                '40°24\'28.64"N', '40°26\'43.97"N', '40°28\'41.64"N',
                '40°23\'17.33"N', '40°23\'53.17"N', '40°26\'23.61"N',
                '40°24\'52"N', '40°27\'56.1"N', '40°22\'22.48"N',
                '40°27\'44.51"N', '40°23\'6.1"N', '40°29\'39.12"N',
                '40°31\'4.97"N', '40°27\'54.90"N', '40°30\'1.97"N']}

# Diccionario con los gases y sus claves en las tablas del Ayuntamiento
claves_gases = {1:'SO2', 6:'CO', 7:'NO', 8:'NO2', 9:'PM2.5', 10:'PM10', 12:'NOx', 14:'O3',
                20:'TOL', 30:'BEN', 35:'EBE', 37:'MXY', 38:'PXY', 39:'OXY', 42:'TCH', 43:'CH4'}


meses = ['ene', 'feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']




In [3]:
def maximun_value(estacion, mes, df, j, year, valor_max):
    '''
    Obtenemos el valor máximo de contaminación de cada estación y cada día. 
    
    Entrada: 
        Número de estación.
        Mes.
        DataFrame limpio.
        Gas.
        Año.
        DataFrame original.
    
    Salida:
        Valores máximos.
        Estacion.
        Gas. 
        Mes. 
        Año.
        Posición de la estación.
    '''
    
    posicion_estacion = estaciones.index(estacion)
    
    for ind in df.index:
        var_y = df.iloc[ind]
        indice_max = np.nanargmax(var_y)
        valor_max['%s' %estaciones_nombres[posicion_estacion]].iloc[ind] = np.nanmax(var_y)
        
        make_directories(valor_max, var_y, j, estacion, mes, year, posicion_estacion, ind)

In [4]:
def make_directories(valor_max, var_y, j, estacion, mes, year, posicion_estacion,ind):
    if os.path.isdir('./Resultados/'):
        pass
    else:
        os.mkdir('./Resultados/')

    if os.path.isdir('./Resultados/Graficas_%s_%s_gas_%s' %(mes, year, j)):
        pass
    else:
        os.mkdir('./Resultados/Graficas_%s_%s_gas_%s' % (mes,year, j))
    
    condition = input('¿Quieres plotear los valors de todos los días las 24 horas? (Y/N): ')
    if condition == 'Y':
        try:    
            plot_diary(var_y, j, estacion, mes, year, posicion_estacion, ind, valor_max)
        except():
            print(logging.error)
    else:
        pass

In [5]:
def plot_diary(var_y, j, estacion, mes, year, posicion_estacion, ind, valor_max):
    '''
    Método para plotterar los valores de todas las horas del día, por estación y día del año.
    Devolverá una gráfica por cada estación y cada día con los 24 valores tomados en cada hora.
    '''
    

    figura = plt.figure(figsize=(16,9))
    x_horas = np.arange(1, 25, 1)
    figura = sns.lineplot(x_horas, var_y, color='#FFA200' , linewidth=3)
    plt.xlabel('Horas', fontsize = 16)
    plt.ylabel('%s (µg/m3)' %j)
    plt.title('%s (%d-%s-%s)'%(estaciones_nombres[posicion_estacion], ind+1, mes, year), fontsize = 20)
    plt.xticks(x_horas)
    plt.yticks(np.linspace(0,var_y.max(),35))
    plt.savefig('./Resultados/Graficas_%s_gas_%s/dia_%d_%s.jpg' 
                %(mes, j, ind+1,estaciones_nombres[posicion_estacion]),bbox_inches='tight')
    time.sleep(1)
    plt.close()
    


In [6]:
def plot_max_monthly (valor_max, mes, year, j):
    
    '''
    Método por el cual creamos gráficas interactivas con los valores máximos de contaminación recogidos en cada
    estación cada día. 
    
    Cada gráfica tiene los valores máximos para cada día de mes de todas las estaciones de Madrid. Así como una
    línea horizontal que marca el valor umbral recogido en los documentos del Ayuntamiento, como valores límite.
    '''
    
    
    if j == 'NOx':
        lim = 400
        uni = '(µg/m3)'
    elif j == 'SO2':
        lim = 500
        uni = '(µg/m3)'
    elif j == 'CO':
        lim = 10
        uni = '(mg/m3)'
    elif j == 'O3':
        lim = 240
        uni = '(µg/m3)'
    else:
        lim = None
        uni = '(µg/m3)'
    valor_max.to_csv('./Resultados/Graficas_%s_%s_gas_%s/tabla_maximos.csv' %(mes, year, j), sep = ';')
    init_notebook_mode(connected = True)
    cf.go_offline()
    valor_max.iplot(hline=lim,
                    annotations=[dict( x=31, y=lim+(0.1*lim), textangle=0,  xref='x', yref='y', 
                                      text="Umbral de alerta", color = '#db4052',  ax=0, ay=0)],
                    title=('Valor máximo de contamicación diaria: %s - %s' %(mes,year)), 
                    xTitle = ('Día del mes'), 
                    yTitle = ('%s %s'%(j, uni)))
    time.sleep(2)


In [7]:
year = input('Introduce el año a estudiar: \n2020 -> 20\n2019 -> 19\n\n\n') 

print('\n\nTenga en cuenta que no todas las estaciones de medición tienen la capacidad de medir todos los gases.\n'
        'Para más información, consultar: https://datos.madrid.es/portal/site/egob/ \n\n')


gas = int(input('A continuación escribe el código asociado al gas que quieres representar: \n\n'
            'SO2 -> 1  ;   CO -> 6  ;   NO -> 7\n' 
            'NO2 -> 8  ;   PM2.5 -> 9 ; PM10 -> 10\n'
            'NOx -> 12  ;  O3 -> 14  ;  TOL -> 20\n'
            'BEN -> 30  ;  EBE -> 35  ; MXY -> 37\n'
            'PXY -> 38  ;  OXY -> 39  ; TCH -> 42\n'
            'CH4 -> 43\n'))

for mes in meses:
    valor_max = pd.DataFrame(columns=estaciones_nombres, index = np.arange(1,32,1))
    j=claves_gases[gas]
    for i in estaciones:
        loc = './Tablas/year_%s' %year
        nombre = 'tabla_corregida_%s_%s_%s.csv'%(mes, j, i)
        df = pd.read_csv(loc+nombre, sep=';')
        df.drop(['Unnamed: 0', 'DIA', 'ESTACION'], axis = 1, inplace=True)
        try:
            maximun_value(i, mes, df, j, year, valor_max)
        except(ValueError):
            pass
    plot_max_monthly (valor_max, mes, year, j)

Introduce el año a estudiar: 
19
20


19


Tenga en cuenta que no todas las estaciones de medición tienen la capacidad de medir todos los gases.
Para más información, consultar: https://datos.madrid.es/portal/site/egob/ 


A continuación escribe el código asociado al gas que quieres representar: 

SO2 -> 1  ;   CO -> 6  ;   NO -> 7
NO2 -> 8  ;   PM2.5 -> 9 ; PM10 -> 10
NOx -> 12  ;  O3 -> 14  ;  TOL -> 20
BEN -> 30  ;  EBE -> 35  ; MXY -> 37
PXY -> 38  ;  OXY -> 39  ; TCH -> 42
CH4 -> 43
12
